In [ ]:
import torch
import matplotlib.pyplot as plt
import math
import numpy
import seaborn as sns
import numpy as np
from pathlib import Path

config = {
    'freq':2e6, # 0 to 100 MHz
    'snr': 1, # mf snr
    'samples': 8192,
    'sample_freq': 200e6,
    'number_signals': 1,
    'number_noise': 1,
    'number_channel': 2, # real, imag.

}

def GenerateSinusoid(config):

    t = torch.arange(0, config['samples'], 1) * 1 / config['sample_freq']

    signal = 1 * torch.exp(1j * 2 * math.pi * t * config['freq'])

    return signal

def CalculateNoiseVar(signal, config):

    var = (torch.vdot(signal, signal) / (config['snr'] ** 2)) / config['samples']

    return var

def GenerateDataTensor(signal, config):

    shape = (
        config['number_signals'] + config['number_noise'],
        config['number_channel'],
        config['samples']
        )

    data = torch.zeros(shape, dtype=torch.float)

    data[0:config['number_signals'], 0, :] = signal.real
    data[0:config['number_signals'], 1, :] = signal.imag

    return data

def AddNoise(data, config):

    shape = (
            config['number_signals'] + config['number_noise'],
            config['number_channel'],
            config['samples']
            )
    
    noise = torch.normal(
        mean = 0,
        std = math.sqrt(CalculateNoiseVar(signal, config) / 2),
        size = shape
        )

    return data + noise

def GenNoise(config):

    shape = (
            config['number_signals'] + config['number_noise'],
            config['number_channel'],
            config['samples']
            )
    
    noise = torch.normal(
        mean = 0,
        std = math.sqrt(CalculateNoiseVar(signal, config) / 2),
        size = shape
        )

    return noise

def FFT(signal):
    
    return torch.fft.fft(signal, norm='forward')


In [ ]:
signal = GenerateSinusoid(config)
fft_signal = FFT(signal)

data = GenerateDataTensor(signal, config)
fft_data = GenerateDataTensor(fft_signal, config)

noisy_data = AddNoise(data, config)
noisy_fft = AddNoise(fft_data, config)

time = np.arange(0, 8192, 1) / 200e6

In [ ]:
print(data.shape)

In [ ]:
plt.plot(signal.real)
plt.xlim(0, 200)

In [ ]:
plt.plot(abs(fft_signal))
#plt.xlim(0, 200)

In [ ]:
sns.set_theme(context='talk')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(time, noisy_data[0, 0, :], label='Noise')
ax.plot(time, signal.real, label='Signal')
ax.set_xlabel('Time (sec)')
ax.set_ylabel('Amplitude (AU)')
ax.set_title(f'Test Signal, SNR = {config["snr"]}')
ax.legend(loc=1)

plt.tight_layout()

name = f'220224_test_signal_snr{config["snr"]}.png'
save_path = Path.home()/'group'/'project'/'plots'/'machine_learning'/'dnn'/'triggering'/'toy_sinusoid'/name

plt.savefig(save_path)

In [ ]:
plt.plot(noisy_data[0, 0, :])
plt.plot(signal.real)
#plt.ylim(-2, 2)
#plt.xlim(0, 256)

In [ ]:
plt.plot(abs(torch.fft.fft(noisy_data[0, 0, :] + 1j * noisy_data[0, 1, :])) ** 2)
#plt.plot(abs(torch.fft.fft(data[0, 0, :] + 1j * data[0, 1, :], norm='forward')) ** 2)

In [ ]:
plt.plot(abs(torch.fft.fft(data[0, 0, :] + 1j * data[0, 1, :], norm='forward')) ** 2)

In [ ]:
plt.plot(abs(noisy_fft[0, 0, :]) ** 2)
plt.plot(abs(fft_signal.real + 1j * fft_signal.imag) ** 2)
plt.xlim(0, 100)

In [ ]:
noise_tensor = GenNoise(config)
noise_tensor = noise_tensor[0, 0, :] + 1j * noise_tensor[0, 1, :]

In [ ]:
plt.plot(abs(torch.fft.fft(noise_tensor, norm='forward')) ** 2)
plt.plot(abs(torch.fft.fft(data[0, 0, :] + 1j * data[0, 1, :], norm='forward')) ** 2)
#plt.xlim(0, 100)